## Assignment 2

In [1]:
from functools import reduce
import itertools
import re
import time

In [2]:
class CSP():
    """This class describes finite-domain Constraint Satisfaction Problems.
    A CSP is specified by the following inputs:
        variables   A list of variables; each is atomic (e.g. int or string).
        domains     A dict of {var:[possible_value, ...]} entries.
        neighbors   A dict of {var:[var,...]} that for each variable lists
                    the other variables that participate in constraints.
        constraints A function f(A, a, B, b) that returns true if neighbors
                    A, B satisfy the constraint when they have values A=a, B=b

    In the textbook and in most mathematical definitions, the
    constraints are specified as explicit pairs of allowable values,
    but the formulation here is easier to express and more compact for
    most cases. (For example, the n-Queens problem can be represented
    in O(n) space using this notation, instead of O(N^4) for the
    explicit representation.) In terms of describing the CSP as a
    problem, that's all there is.

    However, the class also supports data structures and methods that help you
    solve CSPs by calling a search function on the CSP. Methods and slots are
    as follows, where the argument 'a' represents an assignment, which is a
    dict of {var:val} entries:
        assign(var, val, a)     Assign a[var] = val; do other bookkeeping
        unassign(var, a)        Do del a[var], plus other bookkeeping
        nconflicts(var, val, a) Return the number of other variables that
                                conflict with var=val
        curr_domains[var]       Slot: remaining consistent values for var
                                Used by constraint propagation routines.
    The following methods are used only by graph_search and tree_search:
        actions(state)          Return a list of actions
        result(state, action)   Return a successor of state
        goal_test(state)        Return true if all constraints satisfied
    The following are just for debugging purposes:
        nassigns                Slot: tracks the number of assignments made
        display(a)              Print a human-readable representation
    """

    def __init__(self, variables, domains, neighbors, constraints):
        """Construct a CSP problem. If variables is empty, it becomes domains.keys()."""
        variables = variables or list(domains.keys())

        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraints = constraints
        self.initial = ()
        self.curr_domains = None
        self.nassigns = 0

    def assign(self, var, val, assignment):
        """Add {var: val} to assignment; Discard the old value if any."""
        assignment[var] = val
        self.nassigns += 1

    def unassign(self, var, assignment):
        """Remove {var: val} from assignment.
        DO NOT call this if you are changing a variable to a new value;
        just call assign for that."""
        if var in assignment:
            del assignment[var]

    def nconflicts(self, var, val, assignment):
        """Return the number of conflicts var=val has with other variables."""
        def conflict(var2):
            return (var2 in assignment and
                    not self.constraints(var, val, var2, assignment[var2]))
        return count(conflict(v) for v in self.neighbors[var])

    def display(self, assignment):
        """Show a human-readable representation of the CSP."""
        print('CSP:', self, 'with assignment:', assignment)

    # These methods are for the tree and graph-search interface:

    def actions(self, state):
        """Return a list of applicable actions: nonconflicting
        assignments to an unassigned variable."""
        if len(state) == len(self.variables):
            return []
        else:
            assignment = dict(state)
            var = first([v for v in self.variables if v not in assignment])
            return [(var, val) for val in self.domains[var]
                    if self.nconflicts(var, val, assignment) == 0]

    def result(self, state, action):
        """Perform an action and return the new state."""
        (var, val) = action
        return state + ((var, val),)

    def goal_test(self, state):
        """The goal is to assign all variables, with all constraints satisfied."""
        assignment = dict(state)
        return (len(assignment) == len(self.variables)
                and all(self.nconflicts(variables, assignment[variables], assignment) == 0
                        for variables in self.variables))

    def support_pruning(self):
        """Make sure we can prune values from domains."""
        if self.curr_domains is None:
            self.curr_domains = {v: list(self.domains[v]) for v in self.variables}

    def suppose(self, var, value):
        """Start accumulating inferences from assuming var=value."""
        self.support_pruning()
        removals = [(var, a) for a in self.curr_domains[var] if a != value]
        self.curr_domains[var] = [value]
        return removals

    def prune(self, var, value, removals):
        """Rule out var=value."""
        self.curr_domains[var].remove(value)
        if removals is not None:
            removals.append((var, value))

    def infer_assignment(self):
        """Return the partial assignment implied by the current inferences."""
        self.support_pruning()
        return {v: self.curr_domains[v][0]
                for v in self.variables if 1 == len(self.curr_domains[v])}

    def restore(self, removals):
        """Undo a supposition and all inferences from it."""
        for B, b in removals:
            self.curr_domains[B].append(b)

In [3]:
def first(iterable, default=None):
    """Return the first element of an iterable or the next element of a generator; or default."""
    try:
        return iterable[0]
    except IndexError:
        return default
    except TypeError:
        return next(iterable, default)

def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(bool(x) for x in seq)

def different_values_constraint(A, a, B, b):
    return a != b

def flatten(seqs):
    return sum(seqs, [])

_R3 = list(range(3))
_CELL = itertools.count().__next__
_BGRID = [[[[_CELL() for x in _R3] for y in _R3] for bx in _R3] for by in _R3]
_BOXES = flatten([list(map(flatten, brow)) for brow in _BGRID])
_ROWS = flatten([list(map(flatten, zip(*brow))) for brow in _BGRID])
_COLS = list(zip(*_ROWS))

_NEIGHBORS = {v: set() for v in flatten(_ROWS)}
for unit in map(set, _BOXES + _ROWS + _COLS):
    for v in unit:
        _NEIGHBORS[v].update(unit - {v})

### Sudoku Class:

In [4]:
class Sudoku(CSP):
   
    R3 = _R3
    Cell = _CELL
    bgrid = _BGRID
    boxes = _BOXES
    rows = _ROWS
    cols = _COLS
    neighbors = _NEIGHBORS

    def __init__(self, grid):
        """Build a Sudoku problem from a string representing the grid:
        the digits 1-9 denote a filled cell, '.' or '0' an empty one;
        other characters are ignored."""
        squares = iter(re.findall(r'\d|\.', grid))
        domains = {var: [ch] if ch in '123456789' else '123456789'
                   for var, ch in zip(flatten(self.rows), squares)}
        for _ in squares:
            raise ValueError("Not a Sudoku grid", grid)  # Too many squares
        CSP.__init__(self, None, domains, self.neighbors, different_values_constraint)
        self.support_pruning()

    def display(self, board):
        if isinstance(board,Sudoku):
            assignment = board.infer_assignment()
        elif isinstance(board,dict):
            assignment = board
        else:
            raise TypeError("Type needs to be dict or Suduko, instead type was " + str(type(board)))

        def show_box(box): return [' '.join(map(show_cell, row)) for row in box]

        def show_cell(cell): return str(assignment.get(cell, '.'))

        def abut(lines1, lines2): return list(
            map(' | '.join, list(zip(lines1, lines2))))
        print('\n------+-------+------\n'.join(
            '\n'.join(reduce(
                abut, map(show_box, brow))) for brow in self.bgrid))

### Solver:

In [5]:
class Solver:
    def AC3(self, csp, queue=None, removals=None):
        # Establish queue
        if not queue:
            queue = [Neighbors for Variable in csp.curr_domains for Neighbors in self.get_queue(csp, Variable)]
        # Loop through queue until empty
        while queue:
            Xi, Xj = queue[0][0], queue.pop(0)[1]
            if self.revise(csp, Xi, Xj, removals):
                if not csp.curr_domains[Xi]: return False
                for Domain in csp.curr_domains:
                    if Xi in csp.neighbors[Domain] and (Domain, Xi) not in queue and Domain != Xj: 
                        queue.append((Domain, Xi))  
        return True

    def revise(self, csp, Xi, Xj, removals):
        Revised = False
        for XiValue in csp.curr_domains[Xi]:
            constraintSatisfied = [csp.constraints(csp.curr_domains[Xi], XiValue, csp.curr_domains[Xj], XjValue) for XjValue in csp.curr_domains[Xj]]
            if not count(constraintSatisfied):
                csp.prune(Xi, XiValue, removals)
                Revised = True
        return Revised

    ''' recursive call '''
    def backtracking_search(self, csp):
        return self.backtrack({}, csp)

    def backtrack(self, assignment, csp):
        if csp.goal_test(assignment): return assignment
        
        Variable = self.select_unassigned_variable(assignment, csp)
        for Value in self.order_domain_values(Variable, assignment, csp):
            Conflicts = csp.nconflicts(Variable, Value, assignment)
            
            if not Conflicts:
                csp.assign(Variable, Value, assignment)
                csp.suppose(Variable, Value)
                Result = self.backtrack(assignment, csp)
                if Result is not None:
                    return Result
                csp.unassign(Variable, assignment)
                
        return None

    # START: DEFINED ALREADY
    def select_unassigned_variable(self, assignment, csp):
        unassigned = []
        for i in csp.variables:
            if i not in assignment:
                unassigned.append(i)

        sorted_unassigned = sorted(unassigned, key=lambda var: len(csp.domains[var]))
        return sorted_unassigned[0]

    def order_domain_values(self, var, assignment, csp):
        return sorted(csp.domains[var], key=lambda val: csp.nconflicts(var, val, assignment))

    def get_queue(self, csp, var):
        queue = []
        for i in csp.neighbors[var]:
            queue.append((i, var))
        return queue
    # END: DEFINED ALREADY

In [6]:
'''
Some board test cases, each string is a flat enumeration of all the board positions
where . indicates an unfilled location
Impossible: 123456789.........123456789123456789123456789123456789123456789123456789123456789
Easy ..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..
Easy ...7.46.3..38...51.1.9.327..34...76....6.8....62...98..473.6.1.68...13..3.12.5...
Difficult ..5...1.3....2.........176.7.49....1...8.4...3....7..8.3.5....2....9....4.6...9..
'''

# board = sudoku.Sudoku('12.456789.........12.45678912.45678912.45678912.45678912.45678912.45678912.456789')
''' Easy 1 '''
#board = Sudoku('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
''' Easy 2 '''
#board = Sudoku('...7.46.3..38...51.1.9.327..34...76....6.8....62...98..473.6.1.68...13..3.12.5...')
''' Difficult '''
board = Sudoku('..5...1.3....2.........176.7.49....1...8.4...3....7..8.3.5....2....9....4.6...9..')
''' Arto Inkala Difficult '''
#board = Sudoku('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..')
''' Evil Sudoku '''
#board = Sudoku('.7...2.686..5...13......7....6...5..4..........2.7.8.......14.7..32.6....1...5...')
''' Impossible '''
#board = Sudoku('123456789.........123456789123456789123456789123456789123456789123456789123456789')
# Accessing the board as a csp, i.e. display the variable and domains
# See the extra document for exapmles of how to use the  CSP class

# Display this nonsensical board
board.display(board)

# Show the "flat" variables
print(board.variables)

# show the domeians (curr_domains beocmes populated by infer_assignment())
print(board.curr_domains)

'''You'll need to manipulate the CSP domains and variables, so here are some exampels'''

# this is a list of (variable, domain value) pairs that you can use to keep track
# # of what has been removed from the current domains
removals = []

# #show domains for variable 3
print("Domain for 3: " + str(board.curr_domains[3]))
# #remove the possible value '8' form domain 3
# #not the differences int key for the first dictionary and the string keys

if '8' in board.curr_domains[3]:
    board.prune(3, '8', removals)  # This line may not work if the domain for 3 does not contain "8"

print("Domain for 3: " + str(board.curr_domains[3]))
print("Removal List: " + str(removals))

# Prune some more
print("Domain for 23: " + str(board.curr_domains[23]))
if '1' in board.curr_domains[23]:
    board.prune(23, '1', removals)
if '2' in board.curr_domains[23]:
    board.prune(23, '2', removals)
if '3' in board.curr_domains[23]:
    board.prune(23, '3', removals)
print("Domain for 23: " + str(board.curr_domains[23]))
print("Removal List: " + str(removals))

# ooopes took away too muche! Restore removals
board.restore(removals)
print("Domain for 23: " + str(board.curr_domains[23]))

# For assigning variables use a dictionary like
assignment = {}
board.assign(23, '8', assignment)
# once all the variables are assigned, you can use goal_test()

# find the neighbors of a variable
print("Neighbors of 0: " + str(board.neighbors[0]))

# check for a constraint, need to plug in a specific var,val, var val combination
# since 0 and 1 and neighbors, they should be different values
print(board.constraints(0, '0', 1, '0'))  # should be false
print(board.constraints(0, '0', 1, '1'))  # should be true i.e. not a constraint

'''to check your implementatios:'''

# AC3 should return false for impossible example above
sol = Solver()
start = time.perf_counter()
print(sol.AC3(board))
print("\n" + "time: " + str(time.perf_counter() - start))
board.display(board)

# backtracking search usage example
start = time.perf_counter()
sol.backtracking_search(board)
print("\n" + "time: " + str(time.perf_counter() - start))
board.display(board)

. . 5 | . . . | 1 . 3
. . . | . 2 . | . . .
. . . | . . 1 | 7 6 .
------+-------+------
7 . 4 | 9 . . | . . 1
. . . | 8 . 4 | . . .
3 . . | . . 7 | . . 8
------+-------+------
. 3 . | 5 . . | . . 2
. . . | . 9 . | . . .
4 . 6 | . . . | 9 . .
[0, 1, 2, 9, 10, 11, 18, 19, 20, 3, 4, 5, 12, 13, 14, 21, 22, 23, 6, 7, 8, 15, 16, 17, 24, 25, 26, 27, 28, 29, 36, 37, 38, 45, 46, 47, 30, 31, 32, 39, 40, 41, 48, 49, 50, 33, 34, 35, 42, 43, 44, 51, 52, 53, 54, 55, 56, 63, 64, 65, 72, 73, 74, 57, 58, 59, 66, 67, 68, 75, 76, 77, 60, 61, 62, 69, 70, 71, 78, 79, 80]
{0: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 1: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 2: ['5'], 9: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 10: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 11: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 18: ['1'], 19: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 20: ['3'], 3: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 4: ['1', '2', '3', '4', '5', '6', '7', '8', '9'], 5: ['1'